In [ ]:
from __future__ import print_function, division
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from scipy.ndimage import *
import pylab
from scipy.stats import *
from math import *
from windrose import WindroseAxes

from scipy.ndimage import label, median_filter
from skimage.morphology import square

In [ ]:
def find_sub_directories(rootdir):
    
    sub_directories = []
    
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        
        if os.path.isdir(d):
            sub_directories.append(d)
            
        else:
            continue
            
    return(sub_directories)

In [ ]:
TOP_PATH = r'/Users/ryan/Desktop/2018_09_21/SMS_2018_0921_1047_1_goodCrop/Retardance'

In [ ]:
SAVE_PATH = os.makedirs(os.path.join(TOP_PATH, 'round 2'))
SAVE_PATH = os.path.join(TOP_PATH, 'round 2')

In [ ]:
## for running the second round of percentile based cleaning
#IMGS = tuple(np.load(os.path.join(TOP_PATH, 'INDEXED test.npy'), allow_pickle=True))
#NAMES = tuple(np.load(os.path.join(TOP_PATH, 'NAMES test.npy'), allow_pickle=True))
#print(len(IMGS), len(NAMES))

In [ ]:
STR_IDENTIFIER = '.tif'
FILE_TYPE = '.tif'
PROCESS = 'float'

FILES = os.listdir(TOP_PATH)
print(FILES)
STR_TRIM = len(FILE_TYPE)

IMGS = []
NAMES = []
for i in FILES:
    if i.endswith(STR_IDENTIFIER):
        NAMES.append(i[:-STR_TRIM])
        if 'Retardance' in i:
            IMGS.append((io.imread(os.path.join(TOP_PATH,i)))) # retardance
        elif 'Orientation' in i:
            if PROCESS == 'float':
                IMGS.append(np.around((io.imread(os.path.join(TOP_PATH,i))/100).astype(np.float), decimals=3)) # convert array values to degrees
            elif ROCESS == 'int':
                IMGS.append((io.imread(os.path.join(TOP_PATH,i))/100).astype(np.uint8)) # convert array values to degrees
            else:
                print('preprocessing info incomplete')
        else:
            print('unknown image type')
    else:
        continue
NAMES = tuple(NAMES)
IMGS = tuple(IMGS)

print(NAMES)

for i in IMGS:
    plt.imshow(i, cmap = 'hsv')
    plt.show()

In [ ]:
PERCENTILES, PAIRS, FILT_IMGS, PX_CT, OB_CT, L_IMGS = [], [], [], [], [], []

for i in IMGS:
    
    t = tuple(np.percentile(i, np.arange(0,101,10)).astype(float)) #based on frequency of angles in image
    #t = tuple(np.arange(0,181,2).astype(int)) #fixed angle bin width
    #t = tuple(np.arange(0,np.amax(i),10).astype(int)) #fixed angle bin width retardance
    PERCENTILES.append(t)
    p = tuple([(t[j], t[j+1]) for j in range(len(t)-1)])
    PAIRS.append(p)
    
    f_i = []
    px = []
    ob = []
    l_i = []
    
    for j in p:
        
        a = np.copy(i)
        a[i <= j[0]] = 0
        #a[i < j[0]] = 0
        a[i > j[1]] = 0
        f_i.append(a)
        
        l = label(binary_opening(a, structure=square(3)), structure=square(3))
        l_i.append(l[0])
        ob.append(l[1])
        px.append(np.sum(a.astype(bool).astype(int)))
        
    FILT_IMGS.append(tuple(f_i))
    PX_CT.append(tuple(px))
    OB_CT.append(tuple(ob))
    L_IMGS.append(tuple(l_i))
    
PERCENTILES, PAIRS, FILT_IMGS, PX_CT, OB_CT, L_IMGS = (tuple(PERCENTILES), tuple(PAIRS), tuple(FILT_IMGS), 
                                                       tuple(PX_CT), tuple(OB_CT), tuple(L_IMGS))

In [ ]:
SIZES = []
for i in range(len(L_IMGS)):
    print('begin loop:', i)
    l = L_IMGS[i]
    s_1 = []
    
    for j in range(len(l)):
        print('   --> subloop:', j)
        current = l[j]
        ids = np.arange(1,OB_CT[i][j]+1,1)
        s_2 = []
        
        for k in ids:
            s_2.append(np.where(current.flatten() == k)[0].shape[0])
            
        s_2 = np.concatenate((np.reshape(ids, (ids.shape[0], 1)), np.reshape(s_2, (len(s_2), 1))), axis = 1)
        s_1.append(s_2)
    
    SIZES.append(tuple(s_1))
    print('       --> complete loop:', i)
    print('')
SIZES = tuple(SIZES)

In [ ]:
MASKS, INDEXED, TISSUES = [], [], []
for i in range(len(L_IMGS)):
    
    stack = np.sum(np.stack(L_IMGS[i]), axis = 0)
    print('mask')
    plt.imshow(stack.astype(bool))
    plt.show()
    MASKS.append(stack.astype(np.uint8))
    
    a = np.copy(IMGS[i])
    a[stack == False] = 0
    print('indexed back to original')
    plt.imshow(a)
    plt.show()
    INDEXED.append(a)
    
    TISSUES.append(label(a, structure=square(3)))
    print('labeled')
    plt.imshow(TISSUES[-1][0])
    plt.show()

MASKS, INDEXED = (tuple(MASKS), tuple(INDEXED))

In [ ]:
np.save(os.path.join(SAVE_PATH, 'NAMES test.npy'), NAMES, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'IMGS test.npy'), IMGS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'PERCENTILES test.npy'), PERCENTILES, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'PAIRS test.npy'), PAIRS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'FILT_IMGS test.npy'), FILT_IMGS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'PX_CT test.npy'), PX_CT, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'OB_CT test.npy'), OB_CT, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'L_IMGS test.npy'), L_IMGS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'SIZES test.npy'), SIZES, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'MASKS test.npy'), MASKS, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'INDEXED test.npy'), INDEXED, allow_pickle=True)
np.save(os.path.join(SAVE_PATH, 'TISSUES test.npy'), TISSUES, allow_pickle=True)